#### 导入所需的第三方库

In [1]:
import pandas as pd 
import numpy as np 
import re
import pymongo

from pyecharts.charts import Pie, Bar, Map, WordCloud,Line,Grid,Scatter,Radar,Page 
from pyecharts import options as opts
from pyecharts.globals import SymbolType
from pyecharts.globals import ThemeType

#### 连接mongodb数据库读取数据

In [2]:
client = pymongo.MongoClient('localhost', 27017)
db = client['data']
job_data = db['info']
data = pd.DataFrame(list(job_data.find()))

#### 预览数据集前五行信息

In [3]:
data.head(5)

,_id,职位名称,公司名称,工作城市,薪资范围,学历要求,公司类型,公司规模,职位类别,工作经验,福利待遇
0,5fe143d01766012a7d73fa28,在家作业/网上兼职/线上操作,中环汇峰环境科技(北京)有限公司,北京,8千-1万,初中及以下,其它,20人以下,技工/操作工,不限,"五险一金,加班补助,全勤奖,绩效奖金"
1,5fe143d01766012a7d73fa2b,送货司机,中融永信(北京)投资管理有限公司,北京,4千-6千,初中及以下,其它,500-999人,技工/操作工,1-3年,"包吃,包住,节日福利,带薪年假,上六休一"
2,5fe143d01766012a7d73fa2e,警卫,美国驻华大使馆,北京,6千-8千,初中及以下,国家机关,100-299人,行政/后勤/文秘,1年以下,"五险一金,加班补助,带薪年假"
3,5fe143d01766012a7d73fa31,Bodyguard,美国驻华大使馆,北京,8千-1万,初中及以下,国家机关,100-299人,社区/居民/家政服务,3-5年,"五险一金,带薪年假,年底双薪,补充医疗保险,定期体检,不加班,周末双休,试用期全额"
4,5fe143d01766012a7d73fa34,在家作业/网上兼职/线上操作,中环汇峰环境科技(北京)有限公司,北京,8千-1万,初中及以下,其它,20人以下,技工/操作工,不限,"五险一金,加班补助,全勤奖,绩效奖金"


#### 删除id列并再次查看

In [4]:
del data['_id']
data.head(5)

,职位名称,公司名称,工作城市,薪资范围,学历要求,公司类型,公司规模,职位类别,工作经验,福利待遇
0,在家作业/网上兼职/线上操作,中环汇峰环境科技(北京)有限公司,北京,8千-1万,初中及以下,其它,20人以下,技工/操作工,不限,"五险一金,加班补助,全勤奖,绩效奖金"
1,送货司机,中融永信(北京)投资管理有限公司,北京,4千-6千,初中及以下,其它,500-999人,技工/操作工,1-3年,"包吃,包住,节日福利,带薪年假,上六休一"
2,警卫,美国驻华大使馆,北京,6千-8千,初中及以下,国家机关,100-299人,行政/后勤/文秘,1年以下,"五险一金,加班补助,带薪年假"
3,Bodyguard,美国驻华大使馆,北京,8千-1万,初中及以下,国家机关,100-299人,社区/居民/家政服务,3-5年,"五险一金,带薪年假,年底双薪,补充医疗保险,定期体检,不加班,周末双休,试用期全额"
4,在家作业/网上兼职/线上操作,中环汇峰环境科技(北京)有限公司,北京,8千-1万,初中及以下,其它,20人以下,技工/操作工,不限,"五险一金,加班补助,全勤奖,绩效奖金"


#### 查看数据集shape属性

In [5]:
data.shape

(302979, 10)

#### 对于重复公司的职位去重

In [6]:
data.drop_duplicates(subset=['公司名称','职位名称'],inplace=True)
data.shape

(116574, 10)

#### 统计空值，有则删除

In [7]:
data.isnull().sum()
# data.dropna()

职位名称    0
公司名称    0
工作城市    0
薪资范围    0
学历要求    0
公司类型    0
公司规模    0
职位类别    0
工作经验    0
福利待遇    0
dtype: int64

#### 统计工作经验发现有空格的存在

In [8]:
data['工作经验'].value_counts()

1-3年     35629
不限       33560
3-5年     21384
5-10年     9452
无经验       6782
          4292
1年以下      3913
10年以上     1562
Name: 工作经验, dtype: int64

#### 先将空格转换为空值,再用dropna删除空值

In [9]:
data['工作经验'].replace('', np.nan, inplace=True)
data.dropna(subset=['工作经验'], inplace=True)

#### 再次查看，确认空格已经删除

In [10]:
data['工作经验'].value_counts()

1-3年     35629
不限       33560
3-5年     21384
5-10年     9452
无经验       6782
1年以下      3913
10年以上     1562
Name: 工作经验, dtype: int64

#### 查看面议字段

In [11]:
temp = data[data['薪资范围'].isin(['面议'])]
temp.head(5)

,职位名称,公司名称,工作城市,薪资范围,学历要求,公司类型,公司规模,职位类别,工作经验,福利待遇
28,正式员工-西餐厨师/(MUJI com北京京东店),无印良品（上海）商业有限公司,北京,面议,初中及以下,外商独资,1000-9999人,烹饪/料理/食品研发,无经验,
72,饮水机维修员,广州屈臣氏食品饮料有限公司,北京,面议,初中及以下,合资,1000-9999人,电子/电器/半导体/仪器仪表,1-3年,"五险一金,年底双薪,带薪年假,补充医疗保险,定期体检,免费班车,交通补助"
668,央企诚聘帮厨（包食宿）,中冶置业集团,北京,面议,初中及以下,国企,500-999人,烹饪/料理/食品研发,1-3年,"定期体检,带薪年假,包吃,包住,交通补助,弹性工作"
2409,店员/营业员,美团（中国大陆地区）,北京,面议,高中,上市公司,10000人以上,商超/酒店/娱乐管理/服务,不限,绩效奖金
2494,弱电维修工,远洋集团,北京,面议,高中,上市公司,10000人以上,土木/建筑/装修/市政工程,1-3年,"五险一金,绩效奖金,带薪年假,包住,餐补,通讯补助"


#### 反选出非面议字段

In [12]:
df = data[~data['薪资范围'].isin(['面议'])] 

#### 查看薪资范围

In [13]:
print(df['薪资范围'].str[1].value_counts())
print('-'*30)
print(df['薪资范围'].str[-1].value_counts())

千    68007
万    21725
.    19016
0      121
2        1
Name: 薪资范围, dtype: int64
------------------------------
万    62956
千    45749
下      157
上        8
Name: 薪资范围, dtype: int64


#### 利用正则表达式提取薪资数据，剔除千、万

In [14]:
df['薪资范围']

0             8千-1万
1             4千-6千
2             6千-8千
3             8千-1万
6           1万-1.5万
            ...    
302963    1.2万-2.4万
302964        3万-5万
302965         1千以下
302966      2.5万-5万
302978      2.5万-3万
Name: 薪资范围, Length: 108870, dtype: object

In [15]:
ptn = re.compile(r'\d+(\.\d+)?(千|万)')
def split_money(x):
    return [eval(i.group().replace('千','*1000').replace('万','*10000')) for i in ptn.finditer(x)]

                    
salary = df['薪资范围'].apply(split_money)
salary

0              [8000, 10000]
1               [4000, 6000]
2               [6000, 8000]
3              [8000, 10000]
6           [10000, 15000.0]
                 ...        
302963    [12000.0, 24000.0]
302964        [30000, 50000]
302965                [1000]
302966      [25000.0, 50000]
302978      [25000.0, 30000]
Name: 薪资范围, Length: 108870, dtype: object

#### 在原数据中添加最大、最小、平均薪水三列，平均薪水是用mean方法得到

In [16]:
df = df.copy()
df.loc[:,'salary_min'] = salary.str[0].fillna(0).astype('int')
df.loc[:,'salary_max'] = salary.str[1].fillna(0).astype('int')
df.loc[:,'salary_mean'] = df[['salary_min','salary_max']].mean(axis=1).astype('int')
df['salary_mean']

0          9000
1          5000
2          7000
3          9000
6         12500
          ...  
302963    18000
302964    40000
302965      500
302966    37500
302978    27500
Name: salary_mean, Length: 108870, dtype: int32

#### 公司人员重构，去除数据中原有格式

In [17]:
def clean_people(x):
    if x == '20-99人':
        return "20-99"
    elif x == '100-299人':
        return "100-299"
    elif x == '300-499人':
        return "300-499"
    elif x == '500-999人':
        return "500-999"
    elif x == '1000-9999人':
        return "1000-9999"
    elif x == '10000人以上':
        return "大于10000"
    elif x == '20人以下':
        return "少于20"
    else:
        return np.nan
         
data['公司规模'] = data['公司规模'].apply(clean_people)
data['公司规模'].value_counts()

100-299      28201
1000-9999    27252
20-99        21871
大于10000      15359
500-999      12807
少于20          3640
300-499       2500
Name: 公司规模, dtype: int64

#### 数据清洗工作到这一步已经完成的差不多了，现在重组一下数据

In [18]:
df = df[['职位名称','公司名称','工作城市','薪资范围','学历要求','salary_min','salary_max',
                   'salary_mean','公司规模','公司类型','职位类别','工作经验','福利待遇']]
df.columns=['工作','公司','工作城市','薪水','学历','最低薪','最高薪',
                '平均薪','公司规模','公司类型','职位类别','工作经验','福利待遇']
df.head(5)

,工作,公司,工作城市,薪水,学历,最低薪,最高薪,平均薪,公司规模,公司类型,职位类别,工作经验,福利待遇
0,在家作业/网上兼职/线上操作,中环汇峰环境科技(北京)有限公司,北京,8千-1万,初中及以下,8000,10000,9000,20人以下,其它,技工/操作工,不限,"五险一金,加班补助,全勤奖,绩效奖金"
1,送货司机,中融永信(北京)投资管理有限公司,北京,4千-6千,初中及以下,4000,6000,5000,500-999人,其它,技工/操作工,1-3年,"包吃,包住,节日福利,带薪年假,上六休一"
2,警卫,美国驻华大使馆,北京,6千-8千,初中及以下,6000,8000,7000,100-299人,国家机关,行政/后勤/文秘,1年以下,"五险一金,加班补助,带薪年假"
3,Bodyguard,美国驻华大使馆,北京,8千-1万,初中及以下,8000,10000,9000,100-299人,国家机关,社区/居民/家政服务,3-5年,"五险一金,带薪年假,年底双薪,补充医疗保险,定期体检,不加班,周末双休,试用期全额"
6,Bodyguard Shift Leader,美国驻华大使馆,北京,1万-1.5万,初中及以下,10000,15000,12500,100-299人,国家机关,社区/居民/家政服务,3-5年,"五险一金,带薪年假,年底双薪,补充医疗保险,定期体检,不加班,周末双休,试用期全额"


#### 企业性质占比


In [19]:
def pie_rosetype(col) -> Pie:
    c = (
        Pie(init_opts=opts.InitOpts(theme=ThemeType.CHALK))
        .add(
            "",
            [list(z) for z in zip(col.index.tolist(), 
                                  col.values.tolist())],
            radius=["30%", "75%"],
            center=["50%", "50%"],
            rosetype="radius",
            label_opts=opts.LabelOpts(is_show=False),
        )

        .set_global_opts(title_opts=opts.TitleOpts(title="企业性质占比"),
                        legend_opts=opts.LegendOpts(
                        orient="vertical", pos_top="20%", pos_left="0.5%"))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    )
    return c
pie_rosetype(df['公司类型'].value_counts()).render("企业性质占比.html")



'E:\\crawler\\Scrapy\\ZhiLian\\ZhiLian\\spiders\\企业性质占比.html'

#### 企业规模分布

In [20]:
from pyecharts.commons.utils import JsCode
c = (
    Bar(init_opts=opts.InitOpts(width="1500px",theme=ThemeType.WONDERLAND))
    .add_xaxis(df['公司规模'].value_counts().index.tolist())
    .add_yaxis("企业数", df['公司规模'].value_counts().values.tolist(), category_gap="60%")
    .set_series_opts(itemstyle_opts={
        "normal": {
            "color": JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                offset: 0,
                color: 'rgba(0, 244, 255, 1)'
            }, {
                offset: 1,
                color: 'rgba(0, 77, 167, 1)'
            }], false)"""),
            "barBorderRadius": [30, 30, 30, 30],
            "shadowColor": 'rgb(0, 160, 221)',
        }})
    .set_global_opts(title_opts=opts.TitleOpts(title="企业规模分布"),
                     xaxis_opts=opts.AxisOpts(name="企业人数区间")))
c.render("企业规模分布.html")

'E:\\crawler\\Scrapy\\ZhiLian\\ZhiLian\\spiders\\企业规模分布.html'

#### 企业性质薪水对比

In [21]:
gp_job_money = df.groupby('公司类型')['平均薪'].mean()
company = gp_job_money.astype(int).sort_values(ascending=False)
"""柱状图"""
bar = (Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
            .add_xaxis(company.index.tolist())
            .add_yaxis("",company.values.tolist() )
            .set_global_opts(
            title_opts=opts.TitleOpts(title="企业性质薪水对比"),
            datazoom_opts=opts.DataZoomOpts(type_="inside"),
            yaxis_opts=opts.AxisOpts(name="薪水"),
            xaxis_opts=opts.AxisOpts(name="",axislabel_opts={"rotate":30})
        )
    )
bar.render("企业性质薪水对比.html")

'E:\\crawler\\Scrapy\\ZhiLian\\ZhiLian\\spiders\\企业性质薪水对比.html'

#### 热门城市工作数量

In [22]:
city = df.loc[df['工作城市']!='异地招聘',:]['工作城市'].value_counts()[:10].index
number = df.loc[df['工作城市']!='异地招聘',:]['工作城市'].value_counts()[:10].values

pie = (
    Pie(init_opts=opts.InitOpts(theme=ThemeType.WONDERLAND))
    .add(
        "",
        [list(z) for z in zip(city.tolist(), 
                              number.tolist())],
        radius=["40%", "75%"],
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="热门城市工作数量"),
        legend_opts=opts.LegendOpts(
            orient="vertical", pos_top="15%", pos_left="2%"
        ),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)
pie.render("热门城市工作数量.html")

'E:\\crawler\\Scrapy\\ZhiLian\\ZhiLian\\spiders\\热门城市工作数量.html'

#### 薪水与学历走势

In [24]:
edu_m_pivot = df.pivot_table(index="学历",
                                 values = ['平均薪'],
                                 aggfunc = {'平均薪':'mean','学历':'count'})
edu_m_pivot.columns = ['公司规模','平均薪']
edu_m_pivot = edu_m_pivot.sort_values(by=['平均薪'],ascending=False)


grid = Grid()
bar = Bar()
grid.theme = ThemeType.PURPLE_PASSION
line = Line()


bar.add_xaxis(edu_m_pivot.index.tolist())
bar.add_yaxis("学历",edu_m_pivot['公司规模'].tolist(),
              label_opts=opts.LabelOpts(is_show=True))

bar.extend_axis(yaxis=opts.AxisOpts(type_="value",
                                     name="薪水",
                                     position="right",                    
                                     axislabel_opts=opts.LabelOpts(formatter="{value}元"),
                                     ))

bar.set_global_opts(yaxis_opts=opts.AxisOpts(
                                            name="学历人数",
                                            type_="value",
                                            axislabel_opts=opts.LabelOpts(formatter="{value}人")
                                            ), 
                    title_opts=opts.TitleOpts("薪水与学历走势"),
                    tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
                   ) 

line.add_xaxis(edu_m_pivot.index.tolist())
line.add_yaxis("平均薪水",edu_m_pivot['平均薪'].astype('int').tolist(),yaxis_index = 1,label_opts=opts.LabelOpts(is_show=False))

# 把line添加到bar上
bar.overlap(line)
grid.add(chart = bar,grid_opts = opts.GridOpts(),is_control_axis_index = True)
grid.render("薪水与学历走势.html")

'E:\\crawler\\Scrapy\\ZhiLian\\ZhiLian\\spiders\\薪水与学历走势.html'

#### 薪水与工作经验走势

In [25]:
jy_m_pivot = df.pivot_table(index="工作经验",
                                 values = ['平均薪'],
                                 aggfunc = {'平均薪':'mean','工作经验':'count'})
jy_m_pivot.columns = ['公司规模','平均薪']
jy_m_pivot = jy_m_pivot.sort_values(by=['平均薪'],ascending=False)


grid = Grid()
bar = Bar()
grid.theme = ThemeType.DARK 
line = Line()


bar.add_xaxis(jy_m_pivot.index.tolist())
bar.add_yaxis("工作经验",jy_m_pivot['公司规模'].tolist(),
              label_opts=opts.LabelOpts(is_show=True))

bar.extend_axis(yaxis=opts.AxisOpts(type_="value",
                                     name="薪水",
                                     position="right",
                                     axislabel_opts=opts.LabelOpts(formatter="{value}元"),
                                     ))

bar.set_global_opts(yaxis_opts=opts.AxisOpts(
                                            name="经验人数",
                                            type_="value",
                                            axislabel_opts=opts.LabelOpts(formatter="{value}人")
                                            ),
                    title_opts=opts.TitleOpts("薪水与工作经验走势"),
                    tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
                   ) # 交叉指向工具

line.add_xaxis(jy_m_pivot.index.tolist())
line.add_yaxis("平均薪水",jy_m_pivot['平均薪'].astype('int').tolist(),yaxis_index = 1,label_opts=opts.LabelOpts(is_show=False))

# 把line添加到bar上
bar.overlap(line)
grid.add(chart = bar,grid_opts = opts.GridOpts(),is_control_axis_index = True)
grid.render('薪水与工作经验走势.html')

'E:\\crawler\\Scrapy\\ZhiLian\\ZhiLian\\spiders\\薪水与工作经验走势.html'

#### 平均薪水区间分布

In [26]:

# 设置切分区域
listBins = [0,1000,5000,7000,8000,9000,10000,12000,15000,18000,20000,5000000]
# 设置切分后对应标签
listLabels = ['1000以下','1000-5000', '5000-7000','7000-8000','8000-9000',
              '9000-10000','10000-12000','12000-15000', '15000-18000',
              '18000-20000','20000以上']
# 分区统计
time_count_list = pd.cut(df['平均薪'].sort_values(),bins=listBins,labels=listLabels,include_lowest=True)
time_count_list
time_count_dict = {i:0 for i in time_count_list}
time_count_dict
for i in time_count_list:
    time_count_dict[i] = time_count_dict.get(i)+1
time_count_dict 

"""柱状图"""
bar = (Bar(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))
            .add_xaxis(list(time_count_dict.keys()))
            .add_yaxis("", list(time_count_dict.values()))
            .set_global_opts(
            title_opts=opts.TitleOpts(title="平均薪水区间分布"),
            yaxis_opts=opts.AxisOpts(name="人数"),
            xaxis_opts=opts.AxisOpts(name="薪水区间",axislabel_opts={"rotate":30})
        )
    )
bar.render("平均薪水区间分布.html")

'E:\\crawler\\Scrapy\\ZhiLian\\ZhiLian\\spiders\\平均薪水区间分布.html'

#### 福利待遇词云图

In [27]:
job_list = data['福利待遇'].tolist()
title_list = []
for i in job_list:
    try:
        for a in i:
            title_list.append(a)
    except Exception as a:
        pass
    
title_ = np.array(title_list[:10000])


tag_list=','.join(title_).split(',')
tags_count=pd.Series(tag_list).value_counts()

wordcloud = (
    WordCloud()
    .add("",
         [list(z) for z in zip(tags_count.index,tags_count)], 
         word_size_range=[50, 200])
    .set_global_opts(title_opts=opts.TitleOpts(title="福利待遇",title_textstyle_opts=opts.TextStyleOpts(font_size=40)))
)
wordcloud.render("福利待遇词云图.html")

'E:\\crawler\\Scrapy\\ZhiLian\\ZhiLian\\spiders\\福利待遇词云图.html'

#### 按平均薪水统计出排名前10的工作岗位

In [28]:
job = df.sort_values(by=['平均薪'],ascending=False).head(10)
top_job = job['工作'].head(10)
top_job

137374    法务总经理-金融行业-上海/北京
67285               政府事务总监
298596         财务副总裁（地产方向）
45366              兼职平面设计师
285252           销售经理（合伙人）
78280           CEO/总裁/总经理
67066              人力资源副总裁
243374       置业顾问/低薪七千起/包住
296136        华中区域副总裁（投融资）
77621             实验室首席科学家
Name: 工作, dtype: object

#### 统计出排名前10的平均薪水

In [29]:
top_money = job['平均薪'].head(10)
top_money

137374    137500
67285     125000
298596    125000
45366     125000
285252    125000
78280     125000
67066     125000
243374    125000
296136    125000
77621     125000
Name: 平均薪, dtype: int32

#### 根据统计结果绘制Top10岗位平均薪水图

In [30]:
bar = (Bar(init_opts=opts.InitOpts(width="1500px",theme=ThemeType.WONDERLAND))
            .add_xaxis(top_job.tolist())
            .add_yaxis("",top_money.tolist() )
            .set_global_opts(
            title_opts=opts.TitleOpts(title="Top10岗位平均薪水"),
            yaxis_opts=opts.AxisOpts(name="工资"),
            xaxis_opts=opts.AxisOpts(name="工作",axislabel_opts={"interval":"0","rotate":12})
        )
    )
bar.render("Top10岗位平均薪水.html")

'E:\\crawler\\Scrapy\\ZhiLian\\ZhiLian\\spiders\\Top10岗位平均薪水.html'